In [3]:

# MOVIE-TITLE BASED RECOMMENDATION SYSTEM
# Content-Based Filtering using TF-IDF + Cosine Similarity

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re


# 1. LOAD MOVIES DATASET
movies = pd.read_csv("movies.csv")    # movieId, title, genres


# 2. CLEAN TITLES
def clean_title(title):
    title = title.lower()
    title = re.sub(r"\(.*?\)", "", title)        # remove year (1995)
    title = re.sub(r"[^a-z0-9 ]", " ", title)    # remove symbols
    title = re.sub(r"\s+", " ", title).strip()   # clean spaces
    return title

movies["title_clean"] = movies["title"].apply(clean_title)

# 3. TF-IDF ON GENRES + TITLES
movies["combined"] = movies["title_clean"] + " " + movies["genres"].fillna("")

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies["combined"])

# Compute similarity
similarity_matrix = cosine_similarity(tfidf_matrix)

# 4. RECOMMENDATION FUNCTION
def recommend_movie(movie_name, top_n=10):

    movie_name = movie_name.lower().strip()

    # Check movie exists
    if movie_name not in movies["title_clean"].values:
        return f" Movie '{movie_name}' not found."

    # find index of movie
    idx = movies[movies["title_clean"] == movie_name].index[0]

    # Get similarity scores
    scores = list(enumerate(similarity_matrix[idx]))

    # Sort: highest similarity first
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # Take top recommended movies (skip index 0 → same movie)
    top_indices = [i[0] for i in scores[1:top_n+1]]

    # Return movie titles
    return movies.iloc[top_indices][["movieId", "title", "genres"]]


# 5. RUN PROGRAM
if __name__ == "__main__":
    print("\n🎬 Movie Recommendation System Ready!")
    user_movie = input("Enter Movie Name: ")

    print("\n Top Recommendations:\n")
    result = recommend_movie(user_movie)

    if isinstance(result, str):
        print(result)
    else:
        print(result.to_string(index=False))



🎬 Movie Recommendation System Ready!

 Top Recommendations:

 movieId                                                        title genres
    2848 Othello (Tragedy of Othello: The Moor of Venice, The) (1952)  Drama
     386                                                S.F.W. (1994)  Drama
    1922                                              Whatever (1998)  Drama
    3289                 Not One Less (Yi ge dou bu neng shao) (1999)  Drama
    4745                                                     O (2001)  Drama
    4765                                                L.I.E. (2001)  Drama
    5788                                        All or Nothing (2002)  Drama
    6382                            Together (Han ni Zai Yiki) (2002)  Drama
    8458                                       To Each His Own (1946)  Drama
   40870                                            C.R.A.Z.Y. (2005)  Drama
